## Example for ReCappable-seq mapping

### 0. Let check the example reads。

In [1]:
# example reads 
!ls -l read1.fastq read2.fastq

-rw-rw-r-- 1 fox fox 27050914 Mar 28 13:17 read1.fastq
-rw-rw-r-- 1 fox fox 27050914 Mar 28 13:18 read2.fastq


In [2]:
# read1, [11 nt UMI]ATAT[first nucleotide]
!head -4 read1.fastq

@LH00376:50:22JMH5LT3:8:1101:3654:1048 1:N:0:TTCCTACT+GCAAGCAA
CNTGATCGCAAATATAGCAGTTACACTGCGGCGGGCGTCTGTTCTAGTGTTTGAGCCGTCGTGCTTCACCGGTCTACCTCAGATCGGAAGAGCACACGTCT
+
I#IIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIII9IIIIIIIIIIII-III9IIIIIIIIIIIIIIIIIIIIIIIIIIIIIII-IIIIIIIIIII


### 1. QC

In [3]:
!cutadapt --version
# 4.1

4.1


In [4]:
!umi_tools --version
# UMI-tools version: 1.1.1

UMI-tools version: 1.1.1


In [5]:
!cutadapt -j 4 -m 30 -q 20 -e 0.25 --max-n 0 -a AGATCGGAAGAGCACACGTC -A ATATN{{11}}AGATCGGAAGAGCGTCGTG -o read1.cutadapt.fastq -p read2.cutadapt.fastq read1.fastq read2.fastq

This is cutadapt 4.1 with Python 3.9.14
Command line parameters: -j 4 -m 30 -q 20 -e 0.25 --max-n 0 -a AGATCGGAAGAGCACACGTC -A ATATN{11}AGATCGGAAGAGCGTCGTG -o read1.cutadapt.fastq -p read2.cutadapt.fastq read1.fastq read2.fastq
Processing paired-end reads on 4 cores ...
Done           00:00:01       100,000 reads @  14.3 µs/read;   4.20 M reads/minute
Finished in 1.45 s (15 µs/read; 4.13 M reads/minute).

=== Summary ===

Total read pairs processed:            100,000
  Read 1 with adapter:                  46,770 (46.8%)
  Read 2 with adapter:                  71,271 (71.3%)

== Read fate breakdown ==
Pairs that were too short:               6,671 (6.7%)
Pairs with too many N:                      55 (0.1%)
Pairs written (passing filters):        93,274 (93.3%)

Total basepairs processed:    20,200,000 bp
  Read 1:    10,100,000 bp
  Read 2:    10,100,000 bp
Quality-trimmed:                 132,592 bp (0.7%)
  Read 1:        58,278 bp
  Read 2:        74,314 bp
Total written (filtered

In [6]:
!umi_tools extract -I read1.cutadapt.fastq -S read1.umi.fastq --read2-in=read2.cutadapt.fastq --read2-out=read2.umi.fastq -p NNNNNNNNNNNNNNN --log=test_reads.log

## 2. Mapping

In [7]:
!hisat2 --version
#2.2.1

/home/fox/Software/hisat2/2.2.1/hisat2-align-s version 2.2.1
64-bit
Built on Nucleus005
Wed Dec  2 16:48:17 CST 2020
Compiler: gcc version 5.4.0 (GCC) 
Options: -O3 -m64 -msse2 -funroll-loops -g3 -DPOPCNT_CAPABILITY -std=c++11
Sizeof {int, long, long long, void*, size_t, off_t}: {4, 8, 8, 8, 8, 8}


In [8]:
!samtools --version
# 1.16.1

samtools 1.16.1
Using htslib 1.16
Copyright (C) 2022 Genome Research Ltd.

Samtools compilation details:
    Features:       build=configure curses=yes 
    CC:             gcc
    CPPFLAGS:       
    CFLAGS:         -Wall -g -O2
    LDFLAGS:        
    HTSDIR:         htslib-1.16
    LIBS:           
    CURSES_LIB:     -lncursesw

HTSlib compilation details:
    Features:       build=configure libcurl=yes S3=yes GCS=yes libdeflate=no lzma=yes bzip2=yes plugins=no htscodecs=1.3.0
    CC:             gcc
    CPPFLAGS:       
    CFLAGS:         -Wall -g -O2 -fvisibility=hidden
    LDFLAGS:        -fvisibility=hidden 

HTSlib URL scheme handlers present:
    built-in:	 preload, data, file
    S3 Multipart Upload:	 s3w, s3w+https, s3w+http
    Amazon S3:	 s3+https, s3+http, s3
    Google Cloud Storage:	 gs+http, gs+https, gs
    libcurl:	 imaps, pop3, http, smb, gopher, sftp, ftps, imap, smtp, smtps, rtsp, scp, ftp, telnet, rtmp, ldap, https, ldaps, smbs, tftp, pop3s, dict
    crypt4gh

For reference genome, I used GRCh38 (Gencode v45)

Please replace `/home/fox/Database/hisat2/GRCh38_Gencode_v45_nochr_correct/GRCh38.primary_assembly.genome.nochr` by your own indexes.

Also I suggest remove rRNA and other ncRNA by mapping the reads to rRNA and ncRNA reference by bowtie2, not included in this notebook.

In [ ]:
!hisat2 -x /home/fox/Database/hisat2/GRCh38_Gencode_v45_nochr_correct/GRCh38.primary_assembly.genome.nochr --no-temp-splicesite --no-discordant --no-mixed --fr --rna-strandness FR -p 6 -1 read1.umi.fastq -2 read2.umi.fastq | samtools view -bS -F 4 -@ 6 -o hisat2.bam

In [ ]:
# output
!samtools view hisat2.bam | head -3

In [ ]:
!samtools sort -o hisat2.sorted.bam hisat2.bam
!samtools index hisat2.sorted.bam

## 3. UMI tools dedup

In [ ]:
!python --version
# Python 3.9.14

In [ ]:
# I have a script to double check whether the UMI sequence ends with ATAT. You can change my code to specify the end of adapter. You can also skip this step.
# usage: python check_UMI.py <in.bam> <out.bam>
!python check_UMI.py hisat2.sorted.bam hisat2.clean.bam 

In [ ]:
!samtools index hisat2.clean.bam

In [ ]:
!umi_tools dedup --paired --chimeric-pairs=discard --unpaired-reads=discard --stdin=hisat2.clean.bam  --log=umi.logs --method=unique > hisat2.clean.dedup.bam

In [ ]:
!samtools index hisat2.clean.dedup.bam

## 4. Get coverage of the 5' ends

In [ ]:
!python BAM_to_BED_5prime.py hisat2.clean.dedup.bam > test.bed

In [ ]:
!head -10 test.bed
# columns:
# chr, position 0-based, position 1-based, placeholder, read number, strand